In [25]:
import pandas as pd
from src.constants import params, Constants
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
nb_claims_name, claim_amount_name = params.get(Constants.NB_CLAIMS), params.get(Constants.CLAIM_AMOUNT)
claim_frequency_name = params.get(Constants.CLAIM_FREQUENCY)
exposure_name = params.get(Constants.EXPOSURE_NAME)

In [3]:
postcode_mapping = pd.read_csv('postal_code_mapping.csv', delimiter=';')

In [4]:
postcode_mapping.head()

,Code,Localite,Longitude,Latitude,Coordonnees,Geom
0,1050,Ixelles,4.381571,50.822285,"50.8222854,4.3815707",NaN
1,1081,Koekelberg,4.325708,50.862263,"50.8622633,4.3257084",NaN
2,1130,Haren,4.412571,50.891966,"50.8919661,4.41257087324",NaN
3,1150,Woluwe-Saint-Pierre,4.443297,50.829243,"50.8292429,4.4432967",NaN
4,1315,Incourt,4.798944,50.691800,"50.6918004,4.798944",NaN


In [5]:
postcode_mapping['Code'] = postcode_mapping['Code'].astype('str')

In [6]:
postcode_mapping_dic = {}
for code, district in zip(postcode_mapping.Code, postcode_mapping.Localite):
    if code not in postcode_mapping:
        postcode_mapping_dic[code]= district

In [7]:
postcode_mapping_dic

{'1050': 'Ixelles',
 '1081': 'Koekelberg',
 '1130': 'Haren',
 '1150': 'Woluwe-Saint-Pierre',
 '1315': 'Piètrebais',
 '1325': 'Longueville',
 '1341': 'Céroux-Mousty',
 '1350': 'Orp-Le-Grand',
 '1360': 'Thorembais-Les-Béguines',
 '1367': 'Autre-Eglise',
 '1370': 'Saint-Remy-Geest',
 '1380': 'Plancenoit',
 '1390': 'Grez-Doiceau',
 '1430': 'Bierghes',
 '1450': 'Saint-Géry',
 '1495': 'Tilly',
 '1500': 'Halle',
 '1501': 'Buizingen',
 '1600': 'Sint-Pieters-Leeuw',
 '1602': 'Vlezenbeek',
 '1670': 'Pepingen',
 '1671': 'Elingen',
 '1700': 'Sint-Ulriks-Kapelle',
 '1740': 'Ternat',
 '1742': 'Sint-Katherina-Lombeek',
 '1745': 'Opwijk',
 '1750': 'Sint-Kwintens-Lennik',
 '1780': 'Wemmel',
 '1785': 'Brussegem',
 '1852': 'Beigem',
 '1910': 'Nederokkerzeel',
 '1982': 'Elewijt',
 '2040': 'Antwerpen',
 '2100': 'Deurne',
 '2200': 'Herentals',
 '2220': 'Heist-Op-Den-Berg',
 '2222': 'Itegem',
 '2235': 'Houtvenne',
 '2240': 'Viersel',
 '2260': 'Zoerle-Parwijs',
 '2320': 'Hoogstraten',
 '2323': 'Wortel',
 '234

In [15]:
df = pd.read_csv(params.get(Constants.URL_LINK_TO_DATA), delimiter = ",")
df['CODPOSS'] = df['CODPOSS'].astype('str')
df.head()

,AGEPH,CODPOSS,duree,lnexpo,nbrtotc,nbrtotan,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66


In [17]:
df['district'] = df.CODPOSS.replace(postcode_mapping_dic)

In [18]:
df

,AGEPH,CODPOSS,duree,lnexpo,nbrtotc,nbrtotan,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,district
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,Bruxelles
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,Bruxelles
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,Bruxelles
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,Schaerbeek
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,Schaerbeek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163652,50,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Once,Private,No,No,MTPL+,<66,Assenede
163653,86,9960,1.000000,0.000000,0,0.000000,0.000000,6-10,Male,Petrol,Once,Private,No,No,MTPL,<66,Assenede
163654,64,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Twice,Private,No,No,MTPL,<66,Assenede
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,Kaprijke


In [19]:
df[claim_frequency_name] = df[nb_claims_name]/df[exposure_name]

In [20]:
avg_claim_per_district = df.groupby('district')[[claim_frequency_name,nb_claims_name, claim_amount_name]].mean().sort_values(by=[claim_frequency_name, claim_amount_name], ascending=False)

In [21]:
avg_claim_per_district

,claim_frequency,nbrtotc,chargtot
district,,,
Zande,1.410228,0.144330,249.435576
Schelle,0.848366,0.052632,65.522825
Sint-Martens-Latem,0.814071,0.137615,206.304724
Reppel,0.524966,0.107692,113.354081
Hoogstraten,0.520018,0.059406,162.819903
...,...,...,...
Manderfeld,0.000000,0.000000,0.000000
Messines,0.000000,0.000000,0.000000
Reuland,0.000000,0.000000,0.000000


## Geomapping

In [26]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
my_map = gpd.read_file('my_map.kml', driver='KML')
my_map

DriverError: my_map.kml: No such file or directory

In [ ]:
# source:https://gadm.org/
# https://egmara-santos.medium.com/using-python-to-check-geographic-coordinates-on-a-kml-map-baab4510a958